<h2 style="color:orange" >PROJECT-200: House price prediction using machine
learning algorithm </h2>

$\color{cyan}{\text{Data Collect and Cleaning}}$

In [ ]:
#import library..
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("Dataset_Bangadesh.csv") #read dataset

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df["area_type"].value_counts() #getting info 

In [ ]:
df1=df.drop(['area_type','society','availability', 'balcony' ],axis='columns') #remove some unnessesary columns

In [ ]:
df1.head()

In [ ]:
df1.isnull().sum() #Null value

In [ ]:
df2=df1.dropna() #remove null value from dataset
df2.isnull().sum()

In [ ]:
df2["size"].unique() 

In [ ]:
df2['bhk'] = df2['size'].apply(lambda x : int(x.split(' ')[0])) #integer only column for BHK

In [ ]:
df2.head(10)

In [ ]:
df2['bhk'].unique() 

In [ ]:
df2[df2.bhk > 20] 

In [ ]:
df2.total_sqft.unique() #checking incorrect data

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True


In [ ]:
df2[~df2['total_sqft'].apply(is_float)].head(15) #check inconsistent data in 'total_sqft'

In [ ]:
def convert_sqft_to_num(x) :
    tokens = x.split('-')
    if len(tokens) == 2 :
        return (float(tokens[0]) + float(tokens[1]))/2
    try :
        return float(x)
    except :
        return None

In [ ]:
df3 = df2
df3['total_sqft'] = df3['total_sqft'].apply(convert_sqft_to_num) # making consistent
df3.head(10)

In [ ]:
df4 = df3
df4['price_per_sqft'] = df4['price']*100000/df4['total_sqft'] #create new column and count price per squarefeet
df4.head()

In [ ]:
len(df4.location.unique()) #location number check

In [ ]:
df4.location = df4.location.apply(lambda x : x.strip()) #cleaning entries
location_state=df4["location"].value_counts() 
location_state

In [ ]:
len(location_state[location_state <= 10]) #number of occurrences <=10

In [ ]:
location_state_less_then_10 = location_state[location_state <= 10] #save smaller occurrences
location_state_less_then_10

In [ ]:
len(df4.location.unique())

In [ ]:
#Rename location name(less then 10) to 'other'
df4.location = df4.location.apply(lambda x: 'other' if x in location_state_less_then_10 else x)
len(df4.location.unique())

In [ ]:
df4.head()

$\color{cyan}{\text{Removing Outliers}}$

In [ ]:
df4[df4.total_sqft/df4.bhk < 300].head() #sqft per bhk(less then 300)

In [ ]:
df4.shape

In [ ]:
#Remove some location(bhk < 300)
df5 = df4[~(df4.total_sqft/df4.bhk < 300)]
df5.shape

In [ ]:
df5.price_per_sqft.describe() #check price per sqft range

In [ ]:
#Remove very high or very low properties(price per sqft)
def rmv_pps_outlier(df) :
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location') :
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduce_df = subdf[(subdf.price_per_sqft > (m - st)) & (subdf.price_per_sqft <= (m+st))]
        df_out = pd.concat([df_out, reduce_df], ignore_index=True)
    return df_out
df6 = rmv_pps_outlier(df5)
df6.shape

In [ ]:
#check pricing abnormalities
def scatter(df, location) :
    bhk2 = df[(df.location == location) & (df.bhk == 2)]
    bhk3 = df[(df.location == location) & (df.bhk == 3)]
    matplotlib.rcParams['figure.figsize'] = (15, 10)
    plt.scatter(bhk2.total_sqft, bhk2.price, color = 'blue', label = '2 BHK', s = 50)
    plt.scatter(bhk3.total_sqft, bhk3.price, color = 'green', marker = '+', label = '3 BHK', s = 50)
    plt.xlabel('Total Square Feet Area')
    plt.ylabel('Price')
    plt.title(location)
    plt.legend()
scatter(df6, 'Alikadam')

In [ ]:
#remove abnormalities
def rmv_bhk_outlier(df) :
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location') :
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk') :
            bhk_stats[bhk] = {
                'mean' : np.mean(bhk_df.price_per_sqft),
                'std' : np.std(bhk_df.price_per_sqft),
                'count' : bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk') :
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count'] > 5 :
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft < (stats['mean'])].index.values)
    return df.drop(exclude_indices, axis = 'index')

In [ ]:
df7 = rmv_bhk_outlier(df6)
df7.shape

In [ ]:
scatter(df7, 'Alikadam')

In [ ]:
#checking distribution
matplotlib.rcParams['figure.figsize'] = (20, 10)
plt.hist(df7.price_per_sqft, rwidth = 0.8)
plt.xlabel('Price Per Square Feet')
plt.ylabel('Count')

In [ ]:
df7.bath.unique()

In [ ]:
df7[df7.bath > 10]

In [ ]:
plt.hist(df7.bath,rwidth=0.8)
plt.xlabel("Number of bathrooms")
plt.ylabel("Count")

In [ ]:
#removing excess bathrooms
df7[df7.bath > df7.bhk+2]

In [ ]:
#removing excess bathrooms
df8 = df7[df7.bath < df7.bhk+2]
df8.shape

In [ ]:
#removing unnecessary column 
df9 = df8.drop(['size', 'price_per_sqft'], axis = 'columns')
df9.head(3)

$\color{cyan}{\text{Machine Learning Model Building}}$

In [ ]:
dumies = pd.get_dummies(df9.location)
dumies.head(3)

In [ ]:
df10 = pd.concat([df9, dumies.drop('other', axis='columns')], axis = 'columns')
df10.head(3)

In [ ]:
df11 = df10.drop('location', axis = 'columns')
df11.head(3)

In [ ]:
X = df11.drop(['price'], axis = 'columns')
X.head()

In [ ]:
y=df11.price
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
cross_val_score(LinearRegression(), X, y, cv=cv)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
def find_best_model_using_gridsearchcv(X,y):
    algos={
        'linear_regression':{
            'model':LinearRegression(),
            'params':{
                'normalize':[True, False]
                }
        },
        'lasso':{
            'model':Lasso(),
            'params':{
                'alpha':[1,2],
                'selection':['random', 'cyclic']
            }
        },
        'decision_tree':{
            'model':DecisionTreeRegressor(),
            'params':{
                'criterion' :['mse','friedman_mse'],
                'splitter':['best','random']
            }
        }
    }
    scores =[]
    cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for algo_name, config in algos.items():
        gs = GridSearchCV(config['model'],config['params'],cv=cv,return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
            })
    return pd.DataFrame(scores,columns=['model','best_score','best_params'])
find_best_model_using_gridsearchcv(X,y)

In [ ]:
X.columns

In [ ]:
def predict_price(location,sqft,bath,bhk):    
    loc_index=np.where(X.columns==location)[0][0]
    x=np.zeros(len(X.columns))
    x[0]=sqft
    x[1]=bath
    x[2]=bhk
    if loc_index >= 0:
        x[loc_index] = 1
    return lr_clf.predict([x])[0]

In [ ]:
predict_price('Kishoreganj', 1600, 3,3)

In [ ]:
import pickle
with open("Bangladesh_home_price.pickle","wb") as f:
    pickle.dump(lr_clf,f)

In [ ]:
import json
columns={
    "data_columns": [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))